# Beispiel Texte

Wir holen uns ein paar Beispielhafte politische Texte und klassifizieren deren Parteizugehörigkeit mit dem besten Modell. ***Herausfinden eines Schwellwertes für Parteizugehörigkeit 'andere'***

Es werden folgende Imports gemacht: 
 - trainiertes SVM Modell
 - TF-IDF Modell

In [68]:
import pickle
import pandas as pd
import spacy
import pdftotext
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load('de_core_news_sm')

## Trainiertes SVM Modell

In [69]:
model_path = 'Models/best_svc.pickle'

with open(model_path, 'rb') as data:
    svc_model = pickle.load(data)

## TF-IDF Objekt

In [70]:
tfidf_path = 'Pickles/tfidf.pickle'
with open(tfidf_path, 'rb') as data:
    tfidf = pickle.load(data)

## Mapping der Kategorien

In [71]:
party_codes = {
    'FDP': 0,
    'CDU': 1,
    'AfD': 2,
    'SPD': 3,
    'Die Grünen': 4,
    'Die Linke': 5
}

### Funktionen zur Umwandlung von Texten in Features

In [72]:
# Definieren der Vorverarbeitungsfunktionen für einzelne Token
def preprocess_token(token):
    # Reduzierung von Token auf ihre kleingeschriebene lemmatisierte Form
    return token.lemma_.strip().lower()


# Überprüfung ob ein Token zulässig ist
# Es werden nur Token zugelassen, die keine Stop-Wörter, keine Satzzeichen oder nicht alphanumerische Zeichen sind
def is_token_allowed(token):
        if (not token or 
            token.is_stop or 
            token.is_punct or 
            not token.is_alpha):
            return False
        return True

def parseDocument(text):
    lowered_text = text.lower()
    document = nlp(lowered_text)
    allowed_token_list = [preprocess_token(token) for token in document if is_token_allowed(token)]
    parsed_text = " ".join(allowed_token_list)
    return parsed_text

def create_features_from_text(text):
    # bla bla
    features_df = pd.DataFrame(columns=['text'])
    features_df.loc[0] = text
    features_df['text_parsed'] = features_df["text"].apply(parseDocument)
    features_df = features_df['text_parsed']
    
    features = tfidf.transform(features_df).toarray()
    
    return features

Funktion zur Umwandlung der Partei Codes in Parteinamen

In [73]:
def get_party_name(party_id):
    for party, id_ in party_codes.items():    
        if id_ == party_id:
            return party

Den kompletten Workflow zusammensetzen

In [74]:
def predict_from_text(text):
    
    features = create_features_from_text(text)
    
    # Predict using the input model
    prediction_svc = svc_model.predict(features)[0]
    prediction_svc_proba = svc_model.predict_proba(features)[0]
    
    # Return result
    party_svc = get_party_name(prediction_svc)
    
    print("The predicted party using the SVM model is %s." %(party_svc) )
    print("The conditional probability is: %a" %(prediction_svc_proba.max()*100))

### Klassifizierung einer neuen Datei

In [75]:
file_path = 'Wahlprogramme leichte Sprache und andere/Die PARTEI_Wahlprogramm.pdf'

with open(file_path, 'rb') as f:
    program_pdf = pdftotext.PDF(f)# Load PDF document
    # Extract all pages text into one string
    program_text = '\n\n'.join(program_pdf)
    predict_from_text(program_text)

The predicted party using the SVM model is AfD.
The conditional probability is: 46.720773666703096
